# 0. Imports

In [26]:
# 1. DataLoading
import pandas as pd

# 2. Pré-processando 'teorTexto'
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer, WordNetLemmatizer
import nltk
import spacy
from tqdm import tqdm

# Baixar recursos necessários para o nltk (se ainda não tiver instalado)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('stem/rslp')
except LookupError:
    nltk.download('rslp')

try:
  nltk.data.find('wordnet')
except LookupError:
  nltk.download('wordnet')



[nltk_data] Downloading package rslp to /home/esdras-
[nltk_data]     daniel/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package wordnet to /home/esdras-
[nltk_data]     daniel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 1. Data Loading

In [74]:
df = pd.read_csv('./data/Dados_13_fev_25 - menor - df_selecionado.csv')
df.head(3)

,teorTexto,setorDestino,tipoAviso,orgaoJulgador,assuntos,documentos,anexos,classeProcesso
0,PJRN - 4ª VEFT - End. 0893977-47.2022.8.20.500...,AF,I,4ª VET N,10536;5952,Petição;I;Diligência;C;expedir mandado;Não ent...,cda;ficha do imóvel;Sinesp Infoseg - 62581BC5-...,1116
1,PJRN - 4ª VEFT - End. 0854836-60.2018.8.20.500...,AF,I,4ª VET N,10536;5952,Petição;I;Certidão;Portaria conjunta;C;Despach...,COMPROVANTE ENDEREÇO;FICHA DO IMÓVEL;CDA;08548...,1116
2,PJRN - 4ª VEFT - End. 0893734-06.2022.8.20.500...,AF,I,4ª VET N,10536;5952,Petição;I;Diligência;I;Certidão de resposta ne...,cda;renach;RENAJUD POSITIVO - NAJAK NAIBY MAUR...,1116


# 2. Pré-processamento de 'teorTexto'

## 2.1 - Funções

In [95]:
# Carregar modelo de NER do spaCy para português
nlp = spacy.load("pt_core_news_sm")

substituicoes = {
        "Vara de Execução Fiscal e Tributária": "VEFT",
        "Vara da Fazenda Pública": "VFP",
        "Vara da Infância e Juventude": "VIJ",
        "Vara Cível": "VC",
        "Vara de Família e Sucessões": "VF e S",
        "Juizado da Fazenda Pública": "JFP",
        "Juizado Especial da Fazenda Pública": "JEFP",
        "Juizado Especial Cível, Criminal e da Fazenda Pública": "JECFP",
        "Gabinete": "Gab.",
        "Des.": "Des.",
        "Juiz": "J.",
        "Presidente": "Pres.",
        "Câmara Cível": "CC",
        "Turma Recursal": "TR",
        "Comarca de Natal": "Com NAT",
        "Comarca de Mossoró": "Com MOS",
        "Comarca de Parelhas": "Com PAR",
        "Comarca de São Miguel": "Com SMG",
        "Comarca de Apodi": "Com APO",
        "Comarca de Touros": "Com TOU",
        "Comarca de Currais Novos": "Com CN",
        "Pleno": "PL"
    }

# Função para pré-processamento do texto
def preprocess_text(text, use_stemming=False, use_lemmatization=False):
    if isinstance(text, float):  # Lidar com possíveis valores NaN
        return ""
    
    for chave, valor in substituicoes.items():
        text = text.replace(chave, valor)

    # 2. Converter para minúsculas
    text = text.lower()
    
    # 1. Substituição de nomes próprios, instituições e datas usando NER
    doc = nlp(text)

    for ent in doc.ents:
        #text = text.replace(ent.text, ent.label_)
        
        if ent.label_ == "PER":  # Pessoas
            text = text.replace(ent.text, "[PESSOA]")
        elif ent.label_ == "ORG":  # Organizações
            text = text.replace(ent.text, "[ORGANIZACAO]")
        elif ent.label_ == "DATE":  # Datas
            text = text.replace(ent.text, "[DATA]")
        

    # 3. Remoção de caracteres especiais, pontuação, etc. (manter números)
    #text = re.sub(r"[^\w\d\s]", " ", text)  # Mantém letras, dígitos e espaços
    #text = re.sub(r"\s+", " ", text)  # Remove espaços extras

    # 4. Remoção de "Stop Words"
    stop_words = set(stopwords.words('portuguese'))
    words = text.split()
    words = [w for w in words if w not in stop_words]
    text = " ".join(words)

    # 5. Substituição de padrões específicos (antes da stemização/lematização)
    text = re.sub(r'PODER\s+JUDICI[ÁA]RIO\s+(?:DO\s+)?ESTADO\s+(?:DO\s+)?RIO\s+GRANDE\s+(?:DO\s+)?NORTE', 'TJRN', text, flags=re.IGNORECASE)
    text = re.sub(r"proc\.?\s*nº", "[PROCESSO]", text, flags=re.IGNORECASE)
    text = re.sub(r"\bexq\b\.?:", "[EXEQUENTE]", text, flags=re.IGNORECASE)
    text = re.sub(r"\bexc\b\.?:", "[EXECUTADO]", text, flags=re.IGNORECASE)
    text = re.sub(r"\bato\s+ordinat[oó]rio\b", "[ATO_ORDINATORIO]", text, flags=re.IGNORECASE)
    text = re.sub(r"\bendere[cç]o\b", "[ENDERECO]", text, flags=re.IGNORECASE)
    text = re.sub(r"(?<!\d)(\d{7}-\d{2}\.\d{4}\.\d{1,2}\.\d{2,4}\.\d{4})(?!\d)", "[NUMERO_PROCESSO]", text) # Números de processo maiores
    text = re.sub(r"(?<!\d)(\d{1,2}/\d{1,2}/\d{2,4})(?!\d)", "[DATA]", text)  # Datas
    text = re.sub(r"(?<!\d)(\d{5}-\d{3})(?!\d)", '[CEP]', text)
    text = re.sub(r"\(\d{2}\)\s?\d{4,5}\d{4}", "[TELEFONE]", text)
    text = re.sub(r"pjrn", "[TJRN]", text, flags=re.IGNORECASE) #tribunal
    text = re.sub(r"\b[rv]eft\b", "[VARA_EXECUCAO]", text, flags=re.IGNORECASE) #vara
     # Substitui várias ocorrências de "CERTIDÃO DE DÍVIDA ATIVA" por um único token
    text = re.sub(r"(certid[ãa]o de d[ií]vida ativa\s*){2,}", "[CDA_MULTIPLE] ", text, flags=re.IGNORECASE)
    text = re.sub(r"certid[ãa]o de d[ií]vida ativa", "[CDA]", text, flags=re.IGNORECASE)
    text = re.sub(r'[^\w\s\[\]]', '', text)

    # 6. Stemização (opcional)
    if use_stemming:
        stemmer = RSLPStemmer()  # Stemmer para português
        words = text.split()
        stemmed_words = [stemmer.stem(w) for w in words]
        text = " ".join(stemmed_words)

    # 7. Lematização (opcional, mutuamente exclusiva com stemming)
    if use_lemmatization:
        lemmatizer = WordNetLemmatizer()  # Para português, idealmente usar outra biblioteca
        words = text.split()
        lemmatized_words = [lemmatizer.lemmatize(w) for w in words]
        text = " ".join(lemmatized_words)
        
    return text

## 2.2 - Processando 'teorTexto'

In [96]:
tqdm.pandas()

df['teorTexto_clean'] = df['teorTexto'].progress_apply(preprocess_text)

100%|██████████| 1500/1500 [01:31<00:00, 16.41it/s]


In [97]:
df['teorTexto_clean'][0]

'[TJRN]  4ª [VARA_EXECUCAO]  end [NUMERO_PROCESSO] [ORGANIZACAO] mn [ORGANIZACAO] marli avelino [ATO_ORDINATORIO] conformidade disposto art 203  4º cpc intimo fazenda pública para prazo 15 quinze dias manifestarse sobre certidão oj natalrn 29 julho 2024 rozana maria silva chsecr'

In [98]:
df['teorTexto'][0]

'PJRN - 4ª VEFT - End. 0893977-47.2022.8.20.5001 Exq.: MN Exc.: MARLI AVELINO ATO ORDINATÓRIO  Em conformidade com o disposto no art. 203, § 4º, do CPC, intimo a Fazenda Pública para, no prazo de 15 (quinze) dias, manifestar-se sobre a certidão do OJ.  Natal/RN, 29 de julho de 2024  ROZANA MARIA DA SILVA ChSecr'

In [106]:
df.drop(columns=['teorTexto','anexos'], inplace=True)

In [108]:
df.to_csv('./data/novos_DataFrames/df_selecionado2.csv')

In [109]:
df

,setorDestino,tipoAviso,orgaoJulgador,assuntos,documentos,classeProcesso,teorTexto_clean
0,AF,I,4ª VET N,10536;5952,Petição;I;Diligência;C;expedir mandado;Não ent...,1116,[TJRN] 4ª [VARA_EXECUCAO] end [NUMERO_PROCES...
1,AF,I,4ª VET N,10536;5952,Petição;I;Certidão;Portaria conjunta;C;Despach...,1116,[TJRN] 4ª [VARA_EXECUCAO] end [NUMERO_PROCES...
2,AF,I,4ª VET N,10536;5952,Petição;I;Diligência;I;Certidão de resposta ne...,1116,[TJRN] 4ª [VARA_EXECUCAO] end [NUMERO_PROCES...
3,AF,I,4ª VET N,10536;5952,Decisão;Petição de suspensão por parcelamento;...,1116,[TJRN] 4ª [VARA_EXECUCAO] end [NUMERO_PROCES...
4,AF,I,4ª VET N,10536;5952,Despacho;Petição;I;Diligência;expedi ofício à ...,1116,[TJRN] 4ª [VARA_EXECUCAO] end [NUMERO_PROCES...
...,...,...,...,...,...,...,...
1495,PF,I,4ª VET N,10536;5952,Petição;I;Petição;Despacho;Certidão de respost...,1116,[TJRN] 4ª [VARA_EXECUCAO] end [NUMERO_PROCES...
1496,PF,I,4ª VET N,10536;5952,Petição;Embargos de Declaração;Sentença;Petiçã...,1116,[TJRN] 4ª [VARA_EXECUCAO] [PROCESSO] [NUMERO...
1497,PF,I,4ª VET N,10536;5952,Impugnação à Exceção de Pré-executividade;I;Pe...,1116,[TJRN] 4ª [VARA_EXECUCAO] [ENDERECO] proc [N...
1498,PF,I,4ª VET N,10536;5952,Certidão;Despacho;Contrarrazões;I;Recurso de a...,1116,pjtjrn apelação cível nº [NUMERO_PROCESSO] ori...
